In [7]:
import torch

In [1]:
!pip install blitz-bayesian-pytorch


     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 4.0MB 735kB/s 
     |████████████████████████████████| 753.4MB 17kB/s 
     |████████████████████████████████| 7.1MB 92kB/s 
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101
  Found existing installation: torchvision 0.6.1+cu101
    Uninstalling torchvision-0.6.1+cu101:
      Successfully uninstalled torchvision-0.6.1+cu101
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [1]:
#!pip install torch torchvision --upgrade
import torch

In [2]:
torch.__version__

'1.5.1'

In [3]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import blitz.models.b_vgg as vgg
import time

In [4]:
import blitz

## A regular linear layer vs Bayesian linear layer

In [5]:
from blitz.modules import BayesianLinear

In [6]:
BLinear = BayesianLinear(in_features=3, out_features=5)

In [7]:
Linear = nn.Linear(in_features=3, out_features=5)

In [13]:
BLinear??


In [8]:
inp = torch.randn(10, 3)

In [9]:
out_ord = Linear(inp)

In [10]:
out_bayes = BLinear(inp)

In [11]:
out_ord

tensor([[ 0.3454, -0.2950,  0.4419,  0.0521,  0.4851],
        [ 0.2570, -0.8503,  0.4663, -0.1323, -0.2645],
        [ 0.2991, -0.2450,  0.1890,  0.0128,  0.5031],
        [-0.3944,  0.1515, -1.0021, -0.9926,  0.7148],
        [-0.1304,  0.0058, -0.4672, -0.6222,  0.6540],
        [-0.1950, -0.9683, -0.4598, -0.7825, -0.6498],
        [-0.1951,  0.2693, -0.2350, -0.7578,  1.0287],
        [ 0.2007, -0.4440,  0.4565, -0.2092,  0.2631],
        [ 0.1854, -0.2867, -0.3955, -0.0973,  0.3348],
        [ 0.1617, -0.1391,  0.4529, -0.2619,  0.6602]],
       grad_fn=<AddmmBackward>)

In [12]:
out_bayes

tensor([[ 0.1295, -0.1778,  0.2153,  0.0857,  0.0920],
        [-0.1096,  0.0165,  0.1473,  0.2175,  0.0528],
        [ 0.1198, -0.1817,  0.1549,  0.0458,  0.1006],
        [ 0.1357, -0.1362, -0.4979, -0.0465, -0.0748],
        [ 0.1417, -0.1553, -0.2414,  0.0015, -0.0176],
        [-0.2803,  0.1639, -0.2940,  0.2025, -0.0414],
        [ 0.2828, -0.2227, -0.2680,  0.0085, -0.0759],
        [ 0.0642, -0.0967,  0.1041,  0.1545,  0.0260],
        [ 0.0292, -0.1399,  0.0094, -0.0152,  0.1183],
        [ 0.1952, -0.1825,  0.0745,  0.1070,  0.0068]],
       grad_fn=<AddmmBackward>)

In [3]:
model_names = sorted(name for name in vgg.__dict__
    if name.islower() and not name.startswith("__")
                     and name.startswith("vgg")
                     and callable(vgg.__dict__[name]))

In [4]:
model_names

['vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19',
 'vgg19_bn']

In [44]:
model = vgg.__dict__['vgg19']()

In [45]:
print(model)

VGG(
  (features): Sequential(
    (0): BayesianConv2d(
      (weight_sampler): TrainableRandomDistribution()
      (bias_sampler): TrainableRandomDistribution()
      (weight_prior_dist): PriorWeightDistribution()
      (bias_prior_dist): PriorWeightDistribution()
    )
    (1): ReLU(inplace=True)
    (2): BayesianConv2d(
      (weight_sampler): TrainableRandomDistribution()
      (bias_sampler): TrainableRandomDistribution()
      (weight_prior_dist): PriorWeightDistribution()
      (bias_prior_dist): PriorWeightDistribution()
    )
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): BayesianConv2d(
      (weight_sampler): TrainableRandomDistribution()
      (bias_sampler): TrainableRandomDistribution()
      (weight_prior_dist): PriorWeightDistribution()
      (bias_prior_dist): PriorWeightDistribution()
    )
    (6): ReLU(inplace=True)
    (7): BayesianConv2d(
      (weight_sampler): TrainableRandomDistribution()

In [46]:
model.features = torch.nn.DataParallel(model.features)

In [47]:
device = torch.device('cuda') 

In [48]:
model.to(device)

VGG(
  (features): DataParallel(
    (module): Sequential(
      (0): BayesianConv2d(
        (weight_sampler): TrainableRandomDistribution()
        (bias_sampler): TrainableRandomDistribution()
        (weight_prior_dist): PriorWeightDistribution()
        (bias_prior_dist): PriorWeightDistribution()
      )
      (1): ReLU(inplace=True)
      (2): BayesianConv2d(
        (weight_sampler): TrainableRandomDistribution()
        (bias_sampler): TrainableRandomDistribution()
        (weight_prior_dist): PriorWeightDistribution()
        (bias_prior_dist): PriorWeightDistribution()
      )
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): BayesianConv2d(
        (weight_sampler): TrainableRandomDistribution()
        (bias_sampler): TrainableRandomDistribution()
        (weight_prior_dist): PriorWeightDistribution()
        (bias_prior_dist): PriorWeightDistribution()
      )
      (6): ReLU(inplace=True)
      

In [49]:
cudnn.benchmark = True

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

In [50]:
cudnn.benchmark = True

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, 4),
        transforms.ToTensor(),
        normalize,
    ]), download=True),
    batch_size=30, shuffle=True,
    num_workers=0, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=30, shuffle=False,
    num_workers=0, pin_memory=True)

Files already downloaded and verified


In [51]:
lr = 1e-3
lr_ = lr
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [52]:
criterion = nn.CrossEntropyLoss()

In [53]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 2 every 30 epochs"""
    lr = lr_ * (0.5 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [54]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    """
    Save the training model
    """
    torch.save(state, filename)

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [55]:
def train(train_loader, model, criterion, optimizer, epoch):
    """
        Run one train epoch
    """
    global freeze

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()

    #if epoch == 3:
     #   model.MOPED_()
      #  freeze = False

    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        #if args.cpu == False:
        input = input.cuda()
        target = target.cuda()

        # compute output
        # if not freeze:
        #     loss = model.sample_elbo(inputs=input,
        #                    labels=target,
        #                    criterion=criterion,
        #                    sample_nbr=3,
			  #              complexity_cost_weight=1/50000)
        # else:
        output = model(input)
        loss = criterion(output, target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        output = model(input)
        output = output.float()
        loss = loss.float()
        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 30 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1))

In [56]:
def validate(val_loader, model, criterion):
    """
    Run evaluation
    """
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        #if args.cpu == False:
        input = input.cuda()
        target = target.cuda()

        # if args.half:
        #     input = input.half()

        # compute output
        with torch.no_grad():
            output = model(input)
            loss = criterion(output, target)

        output = output.float()
        loss = loss.float()

        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 30 == 0:
          print('Test: [{0}/{1}]\t'
                'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                    i, len(val_loader), batch_time=batch_time, loss=losses,
                    top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'
          .format(top1=top1))

    return top1.avg


In [59]:
import os, sys
best_prec1 = 0

save_dir = 'MODEL_PATH/'

if not os.path.exists(save_dir):
        os.makedirs(save_dir)

for epoch in range(50):
        adjust_learning_rate(optimizer, epoch)

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)

        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion)

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
        }, is_best, filename=os.path.join(save_dir, 'checkpoint_{}.tar'.format(epoch)))

Streaming output truncated to the last 5000 lines.
Epoch: [47][41/1667]	Time 0.282 (0.282)	Data 0.009 (0.009)	Loss 2.3064 (2.3022)	Prec@1 3.333 (12.063)
Epoch: [47][42/1667]	Time 0.282 (0.282)	Data 0.009 (0.009)	Loss 2.3015 (2.3022)	Prec@1 6.667 (11.938)
Epoch: [47][43/1667]	Time 0.280 (0.282)	Data 0.009 (0.009)	Loss 2.3027 (2.3022)	Prec@1 10.000 (11.894)
Epoch: [47][44/1667]	Time 0.283 (0.282)	Data 0.011 (0.009)	Loss 2.3035 (2.3022)	Prec@1 6.667 (11.778)
Epoch: [47][45/1667]	Time 0.281 (0.282)	Data 0.009 (0.009)	Loss 2.3034 (2.3022)	Prec@1 10.000 (11.739)
Epoch: [47][46/1667]	Time 0.279 (0.282)	Data 0.009 (0.009)	Loss 2.3023 (2.3022)	Prec@1 13.333 (11.773)
Epoch: [47][47/1667]	Time 0.280 (0.282)	Data 0.009 (0.009)	Loss 2.3025 (2.3022)	Prec@1 10.000 (11.736)
Epoch: [47][48/1667]	Time 0.280 (0.282)	Data 0.009 (0.009)	Loss 2.3017 (2.3022)	Prec@1 10.000 (11.701)
Epoch: [47][49/1667]	Time 0.284 (0.282)	Data 0.011 (0.009)	Loss 2.3020 (2.3022)	Prec@1 6.667 (11.600)
Epoch: [47][50/1667]	Time 